In [1]:
import folium
import os
import json
import pandas as pd
import branca

user = 'Hugo'

In [52]:
#change dir to the location of the data_bourgogne.json file which contains only locations of Burgundy wine
def get_path(user):
    assert user in ['Gabriel','Hugo']
    
    if user == 'Gabriel':
        path ='/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/'
        %run '/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/scripts/function_addlayer.py'
    
    else :
        path = '/Users/'+user+'/Documents/Github/Projet_informatique_ENSAE/'
        %run '/Users/Hugo/Documents/Github/Projet_informatique_ENSAE/cartes/scripts/function_addlayer.py'
    return path

path = get_path(user)
os.chdir(path)

In [53]:
# Open the file
with open('./jsons/data_bourgogne_wtype_wdes.json', encoding = 'utf-8') as json_file:
    data = json.load(json_file)
    
file = open("./cartes/scripts/grandscrus.txt",'r')
grandscrus = file.read().split('\n')

for key in data.keys():
    for gc in grandscrus:
        if data[key]['properties']['appellation'] == gc :
            data[key]['properties']['Grand Cru'] = 1

In [54]:
df = pd.read_csv('./cartes/scripts/cote_beaune.csv', header = None)
df.columns = ['appellation', 'type vin']

In [55]:
appellation = [x.lower() for x in df['appellation']]

In [56]:
data_cote2beaune = {}
data_rouge = {}
data_blanc = {}

for key in data.keys():
    if data[key]['properties']['appellation'].lower() in appellation :
        data_cote2beaune[key] = data[key]
        
for key in data_cote2beaune.keys():
    if data_cote2beaune[key]['properties']['type_vin'] == 'rouge':
        if 'mousseux' not in data_cote2beaune[key]['properties']['appellation'].lower() and "Crémant" not in data_cote2beaune[key]['properties']['appellation'] :
            data_rouge[key] = data_cote2beaune[key]
        
    if data_cote2beaune[key]['properties']['type_vin'] == 'blanc':
        if 'mousseux' not in data_cote2beaune[key]['properties']['appellation'].lower() and "Crémant" not in data_cote2beaune[key]['properties']['appellation']:
            data_blanc[key] = data_cote2beaune[key]

In [57]:
style_redwine = {'fillColor': '#DC1A40', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}
style_whitewine = {'fillColor': '#EAE010', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}

os.chdir(path)

wine_map = folium.Map(location=[47.0333, 4.83333],
                      height=500,
                      tiles = "Stamen Terrain",
                      zoom_start=12,
                      prefer_canvas=True)

# If no image is wanted, call add_layertomap_no_image instead of add_layertomap.
# works exactly as previously.
add_layertomap_logo(data_rouge, style_redwine, user, 'Côte de Beaune Rouge', wine_map)
add_layertomap_logo(data_blanc, style_whitewine,user, 'Côte de Beaune Blanc', wine_map, show = False)
folium.LayerControl().add_to(wine_map)

#wine_map.save(os.path.join('results', 'cote2beaune.html'))
wine_map.save('./cartes/cote2beaune.html')
#display(wine_map)